*NB: This is an auxiliary notebook used to create a nesting tree for the abatement module, cf. A1\_Baseline.ipynb.*

*Read in tech data and arrange:*

In [4]:
tech = techdata_to_tree.load_techcats(pd.read_excel(f['tech'], sheet_name=s))

*Create nesting trees:*

In [5]:
nts = {mm: nesting_tree.nesting_tree(name=mm) for mm in m}

#### 1.1: Input displacing tree

In [6]:
nts["ID"].add_tree(tech["ID"]["upper_categories"], tree_name = 'ID_EC', **{"type_f":type_f['ID_EC']}) # E to C
nts["ID"].add_tree(tech["ID"]["components"], tree_name = "ID_CU", **{"type_f":type_f['ID_CU']}) # C to U
nts["ID"].add_tree(tech["ID"]["techs"], tree_name="ID_TU", **{'type_io': 'output', 'type_f': type_f['ID_TU']}) # U to T
nts["ID"].add_tree(tech["ID"]["techs_inputs"], tree_name="ID_TX", **{'type_f': type_f['ID_TX']}) # T to inputs X
nts["ID"].add_tree(tech["ID"]["basetechs"], tree_name="ID_BU", **{"type_io":"output", "type_f":type_f['ID_BU']}) # baseline technologies
nts["ID"].add_tree(tech["ID"]["basetech_inputs"], tree_name="ID_BX", **{'type_f': type_f['ID_BX']})

#### 1.2: End of pipe tree

In [7]:
if 'EOP' in m:
    nts["EOP"].add_tree(tech["EOP"]["components"], tree_name = "EOP_CU", **{"type_f":type_f['EOP_CU']}) # C to U
    nts["EOP"].add_tree(tech["EOP"]["techs"], tree_name="EOP_TU", **{'type_io': 'output', 'type_f': type_f['EOP_TU']}) # T to U 
    nts["EOP"].add_tree(tech["EOP"]["techs_inputs"], tree_name="EOP_TX", **{'type_f': type_f['EOP_TX']}) # T to inputs X

#### 1.3: Trees related to final goods and emissions account

*Read in tree and data from xlsx:*
* If 'read\_data' is specified in the other trees this is used to read in data from the f['other'] excel file.
* If 'Q2P' is specified the value is used to specify which sheet in f['other'] specifies the Q2P mapping.

In [8]:
for n,tree in other_trees.items():
    nts['ID'].add_tree(f['other'],tree_name=n,**{'sheet': tree['s']})
    if 'read_data' in tree:
        DataBase.GPM_database.merge_dbs(nts['ID'].trees[n].database,excel2py.xl2PM.pm_from_workbook(f['other'],tree['read_data']), 'first')
    if 'Q2P' in tree:
        tech['ID']['Q2P'] = tech['ID']['Q2P'].append(excel2py.xl2PM.pm_from_workbook(f['other'],{tree['Q2P']: 'maps'}).database['Q2P'].vals)

#### 1.4: Namespaces

*Give modules local namespaces:*

In [9]:
standard_sets = ('inp','out','int','wT','map_all','kno_out','kno_inp')
for mm in m:
    namespace = {k: mm + '_' + k for k in standard_sets}
    nts[mm].run_all(**namespace)
    #Also replaces keys with module-specific names, i.e. changes 'inp' to 'ID_inp' in the attributes/keys.
    for std_set in standard_sets:
        if hasattr(nts[mm], std_set):
            setattr(nts[mm], mm + "_" + std_set, getattr(nts[mm], std_set))
            delattr(nts[mm], std_set)

*Add emissions data:*

In [13]:
try:
    DataBase.GPM_database.merge_dbs(nts['ID'].database,excel2py.xl2PM.pm_from_workbook(f['other'], e),'first')
except NameError:
    print('No emissions data were read; default values are supplied by abatement model')

No emissions data were read
